<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="500" alt="cognitiveclass.ai logo"  />
</center>

# Investigation of cryptocurrency exchange rate dynamic (Matic/USD), сalculation and analysis of technical financial indicators, characterizing the cryptocurrency market (ATR, OBV, RSI, AD)

# Lab 6. Knowledge Check

Estimated time needed: **180** minutes

## Objectives

After completing this lab you will be able to:

*   Be confident about your data analysis skills

### **Columns**

* #### `Ts` - the timestamp of the record
* #### `Open` -  the price of the asset at the beginning of the trading period
* #### `High` -  the highest price of the asset during the trading period
* #### `Low` - the lowest price of the asset during the trading period.
* #### `Close` - the price of the asset at the end of the trading period
* #### `Volume` - the total number of shares or contracts of a particular asset that are traded during a given period
* #### `Rec_count` -  the number of individual trades or transactions that have been executed during a given time period
* #### `Avg_price` - the average price at which a particular asset has been bought or sold during a given period
* #### `ATR` - average true range indicator
* #### `OBV` - on-balance volume indicator
* #### `RSI` - relative strength index indicator
* #### `AD` - accumulation / distribution indicator
* #### `BTC_price` - the avarage price from BTC/BUSD dataset 
* #### `BNB_price` - the avarage price from BNB/BUSD dataset
* ##### Additional columns:  'Open_EUR', 'BTC_price_EUR', 'high_EUR', 'High_Normalized', 'Low_Normalized','close_low', 'close_medium', 'close_high', 'rec_count-binned','rec_count_low', 'rec_count_medium', 'rec_count_high'

You will need the following libraries:


In [ ]:
! conda install -c conda-forge ta -y
! conda install scikit-learn -y

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import ta
import warnings
warnings.filterwarnings("ignore")
import seaborn as sns 
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler,PolynomialFeatures
from sklearn.linear_model import Ridge
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split

#set precision 
pd.set_option("display.precision", 2)
#set precision for float
pd.options.display.float_format = '{:.2f}'.format

This function will download the dataset into your browser 


<b>Importing the Data</b>


you will need to download the dataset; if you are running locally, please comment out the following 


In [ ]:
path = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMSkillsNetwork-GPXX053KEN/Lab2DataSet.csv'

Load the csv:


In [ ]:
df = pd.read_csv(path,low_memory=False, index_col=0)
df.index = df.index.astype("datetime64[ns]")
df.head()

We use the method  <code>head()</code>  to display the first 5 columns of the dataframe:


In [ ]:
df.head()

<div class="alert alert-danger alertdanger" style="margin-top: 20px">
<b style="font-size: 2em; font-weight: bold;">Question #1)</b><br>
    <b>Display the data types of each column using the attribute `dtype`.</b>
</div>

In [ ]:
df.dtypes

# Aroon Indicator

The Aroon Indicator is a technical analysis tool used to measure the strength and direction of a trend in a financial market. It consists of two separate lines: the Aroon-Up line and the Aroon-Down line.

The Aroon-Up line measures the number of periods since the most recent high in price, while the Aroon-Down line measures the number of periods since the most recent low in price.

The Aroon Indicator is calculated as follows:

<code>Aroon-Up = ((Number of periods) - (Number of periods since highest high)) / (Number of periods) x 100</code>

<code>Aroon-Down = ((Number of periods) - (Number of periods since lowest low)) / (Number of periods) x 100</code>

The two lines are plotted on a chart, typically ranging from 0 to 100. When the Aroon-Up line is above the Aroon-Down line, it indicates a bullish trend, while a bearish trend is indicated when the Aroon-Down line is above the Aroon-Up line.

Traders use the Aroon Indicator to identify potential trend changes and to help determine when to enter or exit a trade. It is often used in conjunction with other technical analysis tools to confirm trading signals.

<div class="alert alert-danger alertdanger" style="margin-top: 20px">
<h1>Question #2)</h1>  Calculate Aroon Indicator.


In [ ]:
# Calculate the number of periods since the highest high and lowest low
high_period = df['Volume'].rolling(window=len(df), min_periods=1).apply(lambda x: len(df) - x.argmax(), raw=True)
low_period = df['Volume'].rolling(window=len(df), min_periods=1).apply(lambda x: len(df) - x.argmin(), raw=True)

# Calculate the Aroon-Up and Aroon-Down indicators
aroon_up = (len(df) - high_period) / len(df) * 100
aroon_down = (len(df) - low_period) / len(df) * 100

df['Aroon-Up'] = aroon_up
df['Aroon-Down'] = aroon_down

df[["Volume","Aroon-Up","Aroon-Down"]].head(20)

# Williams Percent Range
(WPR) is an indicator used in technical analysis of financial markets to identify overbought or oversold conditions of an asset. The WPR indicator is based on the assumption that the market tends to close near the high during an uptrend and near the low during a downtrend.

The WPR indicator measures the distance between the current closing price and the highest high over a certain number of periods, and expresses it as a percentage of that range. It is calculated using the formula:

 <code>WPR = ((Highest High - Close)/(Highest High - Lowest Low)) x (-100)

The WPR indicator is plotted on a chart as a line that oscillates between -100 and 0. A reading above -20 is considered overbought, while a reading below -80 is considered oversold. Traders and investors use the WPR indicator to identify potential buy or sell signals, as well as to confirm the strength of a trend.

<div class="alert alert-danger alertdanger" style="margin-top: 20px">
    <b style="font-size: 2em; font-weight: bold;">Question #3)</b><br>  Calculate WPR.

In [ ]:
# Calculate the highest high and lowest low over a given period
period = 14
df['Highest High'] = df['High'].rolling(period).max()
df['Lowest Low'] = df['Low'].rolling(period).min()

# Calculate the Williams Percent Range (WPR) using the formula
df['WPR'] = ((df['Highest High'] - df['Close']) / (df['Highest High'] - df['Lowest Low'])) * (-100)
df['WPR'].tail(20)

# The Volume Weighted Average Price (VWAP) 
is a trading indicator that is commonly used by traders and investors to determine the average price of a security over a specific time period. It is calculated by taking the average price of all transactions during the trading day, weighted by the volume of each transaction. This means that transactions with higher volumes have a greater impact on the overall VWAP calculation.

The VWAP is often used by institutional investors and traders to measure the efficiency of their trades, as it provides an indication of whether they were able to buy or sell at a better price than the average market price. Additionally, it can be used to identify potential areas of support or resistance in a security's price, as VWAP can act as a key level where traders may look to enter or exit positions.

<div class="alert alert-danger alertdanger" style="margin-top: 20px">
<b style="font-size: 2em; font-weight: bold;">Question #4)</b><br>  Calculate VWAP.

In [ ]:
# Calculate the typical price
typical_price = (df['High'] + df['Low'] + df['Close']) / 3

# Calculate the VWAP
vwap = (typical_price * df['Volume']).cumsum() / df['Volume'].cumsum()

# Add the VWAP values to the DataFrame
df['VWAP'] = vwap

df[['VWAP']]

<div class="alert alert-danger alertdanger" style="margin-top: 20px">
<b style="font-size: 2em; font-weight: bold;">Question #5)</b><br> Display correlation between new indicators and Volume.


In [ ]:
corr = df[["Volume",'WPR',"Aroon-Up","Aroon-Down","VWAP"]].corr()
corr

<div class="alert alert-danger alertdanger" style="margin-top: 20px">
<b style="font-size: 2em; font-weight: bold;">Question #6)</b><br> Remove NAN elements from the beginning and end of the dataset


In [ ]:
# check for NaN values at the beginning of the dataset
while df.iloc[0].isna().any():
    df = df.drop(df.index[0])

while df.iloc[-1].isna().any():
    df = df.drop(df.index[-1])

df = df.reset_index(drop=True)
df

<div class="alert alert-danger alertdanger" style="margin-top: 20px">
<b style="font-size: 2em; font-weight: bold;">Question #6):</b><br> Show heatmap of correlatoin  between Volume and new indicators


In [ ]:
sns.heatmap(corr)

<div class="alert alert-danger alertdanger" style="margin-top: 20px">
<b style="font-size: 2em; font-weight: bold;">Question #8)</b> Build regplot using "VWAP" as X and Volume as Y


In [ ]:
sns.regplot(x="VWAP", y="Volume", data=df)

<div class="alert alert-danger alertdanger" style="margin-top: 20px">
<b style="font-size: 2em; font-weight: bold;">Question #9)</b><br> Fit a linear regression model to predict the <code>'Volume'</code> using the <code>'Aroon-Down'</code> then calculate $R^{2}$:


In [ ]:
df.dropna(inplace=True)
# Create Multiple Linear Regression
lm = LinearRegression()

# Fit the model
X = df[["Aroon-Down"]]
lm.fit(X, df['Volume'])

# Make prediction
Y_hat = lm.predict(X)


r_squared = r2_score(df['Volume'],Y_hat)
print(f'The R-square value is:{r_squared: .4f}')

<div class="alert alert-danger alertdanger" style="margin-top: 20px">
    <b style="font-size: 2em; font-weight: bold;">Question #10):</b>
Use list of features to predict the 'Volume', split the data into training and testing and determine the  $R^{2}$
  on the test data.

In [ ]:
# Split into train and test subsets
y_data = df['Volume']
x_data=df.drop('Volume',axis=1)
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.10, shuffle=False)

lre=LinearRegression()
r = lre.fit(x_train[['WPR','Aroon-Up','Aroon-Down','VWAP']], y_train)
print("{:.3f}".format(lre.score(x_test[['WPR','Aroon-Up','Aroon-Down','VWAP']], y_test)) )

<div class="alert alert-danger alertdanger" style="margin-top: 20px">
<b style="font-size: 2em; font-weight: bold;"> Question #11)</b><br> Create and fit a Ridge regression object using the training data ('WPR','Aroon-Up','Aroon-Down','VWAP'), setting the alpha parameter to 1 and calculating the $R^{2}$ using the test data. Take a screenshot of your code and the $R^{2}$.

In [ ]:
RigeModel=Ridge()

RigeModel.fit(x_train[['WPR','Aroon-Up','Aroon-Down','VWAP']], y_train)

yhat = RigeModel.predict(x_test[['WPR','Aroon-Up','Aroon-Down','VWAP']])

r_squared = r2_score(y_test, yhat)
print(f'The R-square value is:{r_squared: .4f}')

 <div class="alert alert-danger alertdanger" style="margin-top: 20px">
 <b style="font-size: 2em; font-weight: bold;">Question #12)</b><br> Perform a 2nd order polynomial transform on both the training data and testing data ('WPR','Aroon-Up','Aroon-Down','VWAP'). Create and fit a Ridge regression object using the training data(alpha=1). Calculate the  𝑅2
  utilizing the test data provided. Take a screenshot of your code and the  𝑅2
 .

In [ ]:
pr=PolynomialFeatures(degree=2)

x_train_pr=pr.fit_transform(x_train[['WPR','Aroon-Up','Aroon-Down','VWAP']])
x_test_pr=pr.fit_transform(x_test[['WPR','Aroon-Up','Aroon-Down','VWAP']])

RigeModel=Ridge()

RigeModel.fit(x_train_pr, y_train)

yhat = RigeModel.predict(x_test_pr)

r_squared = r2_score(y_test, yhat)
print(f'The R-square value is:{r_squared: .4f}')

<a href="https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/share-notebooks.html/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkDA0101ENSkillsNetwork20235326-2021-01-01"> CLICK HERE</a> to see how to share your notebook


# **Thank you for completing this lab!**

## Author

<a href="https://author.skills.network/instructors/ostap_liashenyk" target="_blank" >Ostap Liashenyk</a>

<a href="https://author.skills.network/instructors/yaroslav_vyklyuk_2?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkGuidedProjectsIBMSkillsNetworkGPXX0QGDEN2306-2023-01-01">Prof. Yaroslav Vyklyuk, DrSc, PhD</a>

<a href="https://author.skills.network/instructors/mariya_fleychuk?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkGuidedProjectsIBMSkillsNetworkGPXX0QGDEN2306-2023-01-01">Prof. Mariya Fleychuk, DrSc, PhD</a>

## Change Log

| Date (YYYY-MM-DD) | Version | Changed By      | Change Description                                         |
| ----------------- | ------- | ----------------| ---------------------------------------------------------- |
|     2023-04-01    |   1.0   | Ostap Liashenyk | Creation of the lab                                        |

<hr>

## <h3 align="center"> © IBM Corporation 2023. All rights reserved. </h3>
